# How To Use NLP

## 4. MLP를 이용한 Sentiment Analysis
본 강의서 사용된 자료출처: https://github.com/e9t/nsmc

### 4.1 데이터 전처리

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import find
import gensim
#import tensorflow as tf

root_dir = "../data"
train_file = root_dir + '/ratings_train-Copy1.txt'
test_file = root_dir+ '/ratings_train-Copy1.txt'

#### 4.1.1데이터 로딩

In [3]:
train = pd.read_csv(train_file, delimiter = '\t')
test = pd.read_csv(test_file, delimiter='\t')

In [4]:
y_train = train.label.values
y_test = train.label.values

#### 4.1.2 데이터 확인

In [5]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,1
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


#### 라벨: 부정평가(0), 긍정평가(1) 구분

In [6]:
print(y_train)
print(len(y_train))

[0 1 1 ... 0 1 0]
150000


In [7]:
train.loc[0]

id                      9976970
document    아 더빙.. 진짜 짜증나네요 목소리
label                         0
Name: 0, dtype: object

#### 4.1.3 형태소 분석

In [8]:
from konlpy.tag import Twitter, Komoran
from time import strftime
import time
twitter = Twitter()
komoran = Komoran()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
print(train.loc[0, 'document'])
print(twitter.morphs(train.loc[0, 'document'], stem=True, norm=True))
print(komoran.morphs(train.loc[0, 'document']))
print(twitter.pos(train.loc[0, 'document']))
print(komoran.pos(train.loc[0, 'document']))

아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증나다', '목소리']
['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']
[('아', 'Exclamation'), ('더빙', 'Noun'), ('..', 'Punctuation'), ('진짜', 'Noun'), ('짜증나네요', 'Adjective'), ('목소리', 'Noun')]
[('아', 'IC'), ('더빙', 'NNP'), ('.', 'SF'), ('.', 'SF'), ('진짜', 'MAG'), ('짜증', 'NNG'), ('나', 'VV'), ('네요', 'EC'), ('목소리', 'NNG')]


#### 형태소분석-train/test set의 morpheme을 떼어내 저장
train/test_segementation : 각 단어의 형태소만을 떼어내 저장한 것  
train/test_sentence: 1문장씩 들어 있는 것

In [10]:
def segmentation(sequental_data):
    segmentation  = []
    for i in range(len(sequental_data)):
        if isinstance(sequental_data.loc[i, 'document'], float):
            continue
        tokens = komoran.morphs(test.loc[i,'document'])
        segmentation.append(tokens)
    return segmentation

In [11]:
def parsing(segmented_data):
    sentence = []
    for i in range(len(segmented_data)):
        temp = ''
        for j in range(len(segmented_data[i])-1):
            temp += train_segmentation[i][j] + ' '
        temp += segmented_data[i][len(segmented_data[i])-1]
        sentence.append(temp)
    return sentence

In [12]:
#Twitter Speed calc
train_segmentation  = []
start_time = time.time()
for i in range(len(train)):
    if isinstance(train.loc[i, 'document'], float):
        continue
    tokens = twitter.morphs(train.loc[i,'document'], norm=True, stem=True)
    train_segmentation.append(tokens)
print("--- %s seconds ---" %(time.time() - start_time))    

--- 525.0615768432617 seconds ---


In [13]:
#Komoran Speed calc - 코모란을 씁시다
train_segmentation  = []
start_time = time.time()
for i in range(len(train.index)):
    if isinstance(train.loc[i, 'document'], float):
        continue
    tokens = komoran.morphs(train.loc[i,'document'])
    train_segmentation.append(tokens)
print("--- %s seconds ---" %(time.time() - start_time))    

--- 103.13204789161682 seconds ---


In [14]:
train_segmentation = segmentation(train)
test_segmentation  = segmentation(test)

In [15]:
train_sentence = parsing(train_segmentation)
test_sentence = parsing(test_segmentation)

In [17]:
#만들어놓은 list들 저장
np.save('../data/train_segmentation', train_segmentation)
np.save('../data/test_segmentation', test_segmentation)
np.save('../data/train_sentence', train_sentence)
np.save('../data/test_sentence', test_sentence)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [20]:
#이렇게 만들어놓은 np 데이터는 load명령어로 불러옵니다.
train_segmentation = np.load('../data/train_segmentation.npy',allow_pickle=True)
train_sentence = np.load('../data/train_sentence.npy',allow_pickle=True)
test_segmentation = np.load('../data/test_segmentation.npy',allow_pickle=True)
print(train_segmentation[0])

['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']


#### 4.1.4 Word2Vec

In [21]:
?gensim.models.Word2Vec()

Object `gensim.models.Word2Vec()` not found.


In [23]:
model = gensim.models.Word2Vec(train_segmentation,min_count=2, window=5, vector_size=300, sg=1)

In [25]:
#model.save('./data/sentiment.bin')
#모델 불러오기
model = gensim.models.Word2Vec.load('../data/sentiment.bin')

In [26]:
vectors=model.wv

#### 4.1.5 Vectorization

In [27]:
print(train_segmentation[:1],'\n\n',type(train_segmentation),'\n\n')
print(train_sentence[:1],'\n\n',type(train_sentence))

[list(['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'])] 

 <class 'numpy.ndarray'> 


['아 더빙 . . 진짜 짜증 나 네요 목소리'] 

 <class 'numpy.ndarray'>


#### **단어장 생성**

In [28]:
word_dict =[]
for sentences in train_segmentation:
    for segmentation in sentences:
        word_dict.append(segmentation)
print(word_dict[:10])
print(len(word_dict))
print(type(word_dict))

['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리', '흠']
2865517
<class 'list'>


In [29]:
word2num = {word:(index+1) for index,word in enumerate(set(word_dict))} 
num2word = {(index+1):word for index,word in enumerate(set(word_dict))}
print(len(word2num))
print(type(word2num))
for i in range(5):
      print(list(num2word.items())[i])

60400
<class 'dict'>
(1, '화원')
(2, '알랑드롱을')
(3, '그냥너랑봐서좋앗따ㅡ')
(4, '샬리즈테론')
(5, '감동적이다ㅠ')


#### 단어를 Vectorize하기

In [36]:
embedding_vector = [np.zeros(shape=300)]
for index, word in enumerate(word2num.keys()):
    if word not in vectors.key_to_index:
        embedding_vector += [np.zeros(shape=300)] #[np.random.normal(scale=1e-2, size=300)] 이걸 왜 넣었지?
    else:
        embedding_vector += [vectors[word]]

In [37]:
word2num['<UNK>'] = 0
num2word[0] = '<UNK>'

In [38]:
def  sentence2index (segment):
    idx = []
    for sentence in segment:
        temp = []
        for segment in sentence:
            if segment not in word2num.keys():
                segment = '<UNK>'
            temp.append(word2num[segment])
        idx.append(temp)
    return idx

In [39]:
train_idx = sentence2index(train_segmentation)
test_idx = sentence2index(test_segmentation)
train_idx[:1]
test_idx[:1]

[[15710, 12884, 41808, 41808, 28939, 20602, 13046, 10196, 19217]]

In [40]:
def idx2EmbedSum(idxSet, embedding_vector):
    sum_w2v = []
    for idxes in idxSet:
        temp = np.zeros(shape=300)
        for idx in idxes:
            temp += embedding_vector[idx]
        sum_w2v.append(temp)
    return sum_w2v

In [41]:
X_train_w2v = idx2EmbedSum(train_idx, embedding_vector)
X_test_w2v = idx2EmbedSum(test_idx, embedding_vector)

In [42]:
print(len(X_test_w2v[999]))

300


### 4.2 학습및 테스트하기 (Legacy)_2장에 있습니다

### 4.2.1 Tensorflow 기본 동작 원리 -

<img src='../data/pic/3.jpg'>

In [ ]:
output = 2 #0과 1이므로
embedding = 300 #보통 Word2Vec형태의 임베딩, 300차원으로 한다.
hidden = 128 #hidden layer을 지날때의 벡터 차원, 아무렇게 바꿔줘도 된다.
learning_rate = 1e-3 #learning rate, 케이스에 따라 다르긴 한데, 보통 0.01에서 시작한다.
epoch = 10 #몇 번 돌 것인가
batch_size = 64 #batch norm을 할때 한번에 읽어들일 개수

In [ ]:
tf.set_random_seed(0)
#Placeholder
X = tf.placeholder(tf.float32, shape=[None, embedding]) #그래프를 그릴땐, 몇개인지 정의하지 않는다, 300차원
Y = tf.placeholder(tf.int32, shape=None) #정답, 0과 1로만 되어있다

In [ ]:
#Weight, activation
W1 = tf.Variable(tf.truncated_normal([embedding, hidden]))
b1 = tf.Variable(tf.truncated_normal([hidden]))
activate = tf.nn.relu(tf.matmul(X, W1)+b1)#보통 nlp에서 activation function은 hyperbolantic tangent를 쓴다
W2 = tf.Variable(tf.truncated_normal([hidden, output]))
b2 = tf.Variable(tf.truncated_normal([output]))

In [ ]:
#Cost 계산
logits = tf.matmul(activate, W2) + b2
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
prediction = tf.cast(tf.argmax(hypothesis, 1), tf.int32)
correct = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
#Session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train
for epoch in range(epoch):
    total_batch = int(len(y_train) / batch_size)
    cost_avg = 0
    print('< epoch :', (epoch+1), '>')
    for i in range(total_batch):
        if i == (total_batch-1):
            batch_xs = X_train_w2v[(i*batch_size):len(y_train)]
            batch_ys = y_train[(i*batch_size):len(y_train)]
        else:
            batch_xs = X_train_w2v[i*batch_size:(i+1)*batch_size]
            batch_ys = y_train[i*batch_size:(i+1)*batch_size]       
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        cost_avg += cost_val
        if i % 1000 == 999:
            print('%d' % (i+1), 'Cost: ', '{:.3f}'.format(cost_avg/1000))
            cost_avg = 0

In [ ]:
# test
test_batch = int(len(X_test_w2v) / batch_size)
test_acc = 0
for i in range(test_batch):
    if i == (test_batch-1):
        batch_xs = X_test_w2v[(i*batch_size):len(X_test_w2v)]
        batch_ys = y_test[(i*batch_size):y_test.shape[0]]
    else:
        batch_xs = X_test_w2v[i*batch_size:(i+1)*batch_size]
        batch_ys = y_test[i*batch_size:(i+1)*batch_size]       
    acc = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
    test_acc += acc
print('Accuracy: ', '{:.3f}'.format(test_acc/test_batch))